### Data Cleaning Plan
There is an issue with data quality.  We will analyze the Part 1 crimes, which make up the national 'Crime Index'.<br>
<br> 
There are currently 53 unique crime descriptions in the dataset for part 1 crimes.  We will consolidate these down to the 10 Part 1 crimes that are part of the national crime index.<br>
<br>
We will also evaluate part 2 crimes to see if any crimes were mislabeled<br> 

In [1]:
# import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# load the data
data = pd.read_csv('../data/crime_data.csv')

# remove spaces from column names
data.columns = data.columns.str.replace(' ', '')

### Steps to clean data
- Subset the dataset to part1 and part2 crimes
- Create list of keywords that are in Part 1 crimes and iterate through all crime descriptions
- Find crimes with similar categories and consolidate them
- Check for any mislabeled crimes in part 2 dataset and merge with part 1 dataset

In [2]:
# subset the data into part 1 and part 2 crimes
data1 = data[data['Part1-2'] == 1]
data2 = data[data['Part1-2'] == 2]

print('There are {} Part 1 crimes and {} Part 2 crimes'.format(data1.shape[0], data2.shape[0]))

There are 549757 Part 1 crimes and 382383 Part 2 crimes


In [3]:
# create list of words in part 1 crimes list
part1_crimes = ['HOMICIDE', 'RAPE', 'ROBBERY', 'AGGRAVATED ASSAULT', 'BURGLARY', 'LARCENY','THEFT', 'MOTOR','VEHICLE','HUMAN','TRAFFICKING','SEX','ARSON']

# create a crime_category column, look for words in crime desc that match the part1_crimes list
# copy the words that match into new crime category
data1['crime_category'] = data1['CrmCdDesc'].apply(lambda x: ' '.join([word for word in part1_crimes if word in x]))

# review the list of new crime categories
print('There are {} unique crime categories, 5 are redundant and will be consolidated'.format(data1.crime_category.nunique()))
display(data1.crime_category.value_counts().sort_values(ascending=False).to_frame().reset_index())

There are 13 unique crime categories, 5 are redundant and will be consolidated


,crime_category,count
0,THEFT,113103
1,VEHICLE,104442
2,THEFT MOTOR VEHICLE,70197
3,AGGRAVATED ASSAULT,65826
4,BURGLARY,60910
5,BURGLARY VEHICLE,58205
6,ROBBERY,36453
7,,28626
8,RAPE,3983
9,ARSON,2515


In [4]:
# consolidate crimes in MOTOR VEHICLE THEFT category
for i in ['VEHICLE','THEFT MOTOR VEHICLE','MOTOR VEHICLE']:
    data1.loc[data1['crime_category']==i, 'crime_category'] = 'MOTOR VEHICLE THEFT'

# consolidate crimes in BURGLARY category
for i in ['BURGLARY VEHICLE']:
    data1.loc[data1['crime_category']==i, 'crime_category'] = 'BURGLARY'

# consolidate crimes in SEX category
for i in ['SEX']:
    data1.loc[data1['crime_category']==i, 'crime_category'] = 'RAPE'

# review the remaining crimes
display(data1.crime_category.value_counts().sort_values(ascending=False).to_frame().reset_index())

,crime_category,count
0,MOTOR VEHICLE THEFT,176779
1,BURGLARY,119115
2,THEFT,113103
3,AGGRAVATED ASSAULT,65826
4,ROBBERY,36453
5,,28626
6,RAPE,5798
7,ARSON,2515
8,HOMICIDE,1542


### We will explore the blank values in index position 5

In [5]:
print('There are {} crimes that do not fit into the categories, we will consolidate these as well'.format(data1.loc[data1['crime_category']=='']['CrmCdDesc'].value_counts().shape[0]))
display(data1.loc[data1['crime_category']=='']['CrmCdDesc'].value_counts().sort_values(ascending=False).to_frame().reset_index())

There are 17 crimes that do not fit into the categories, we will consolidate these as well


,CrmCdDesc,count
0,BRANDISH WEAPON,14362
1,BIKE - STOLEN,7499
2,PICKPOCKET,2784
3,SHOTS FIRED AT INHABITED DWELLING,1621
4,ASSAULT WITH DEADLY WEAPON ON POLICE OFFICER,1060
5,ORAL COPULATION,697
6,DRIVING WITHOUT OWNER CONSENT (DWOC),167
7,BOAT - STOLEN,132
8,PURSE SNATCHING,107
9,SHOPLIFTING - ATTEMPT,107


In [7]:
# consolidate AGGRAVATED ASSAULT crimes
for i in ['BRANDISH WEAPON','SHOTS FIRED AT INHABITED DWELLING','ASSAULT WITH DEADLY WEAPON ON POLICE OFFICER']:
    data1.loc[data1['CrmCdDesc']==i, 'crime_category'] = 'AGGRAVATED ASSAULT'

# consolidate THEFT crimes
for i in ['BIKE - STOLEN','PICKPOCKET','PURSE SNATCHING','SHOPLIFTING - ATTEMPT','DRUNK ROLL','TILL TAP - PETTY ($950 & UNDER)','PURSE SNATCHING - ATTEMPT','PICKPOCKET, ATTEMPT','BIKE - ATTEMPTED STOLEN']:
    data1.loc[data1['CrmCdDesc']==i, 'crime_category'] = 'THEFT'

# consolidate MOTOR VEHICLE THEFT crimes
for i in ['DRIVING WITHOUT OWNER CONSENT (DWOC)','BOAT - STOLEN']:
    data1.loc[data1['CrmCdDesc']==i, 'crime_category'] = 'MOTOR VEHICLE THEFT'

# consolidate RAPE crimes
data1.loc[data1['CrmCdDesc']=='ORAL COPULATION','crime_category'] = 'RAPE'

# consolidate BURGLARY crimes
data1.loc[data1['CrmCdDesc']=='TRAIN WRECKING','crime_category'] = 'RAPE'

# remove negligent manslaughter as it's not a crime
data1 = data1[data1['CrmCdDesc'] != 'MANSLAUGHTER, NEGLIGENT']

# display updated crime categories
display(data1.crime_category.value_counts().sort_values(ascending=False).to_frame().reset_index())
print('All crime categories were consolidated properly')
print('Now we will review the Part 2 crimes, to see if any crimes were misclassified')

,crime_category,count
0,MOTOR VEHICLE THEFT,177078
1,THEFT,123681
2,BURGLARY,119115
3,AGGRAVATED ASSAULT,82869
4,ROBBERY,36453
5,RAPE,6496
6,ARSON,2515
7,HOMICIDE,1542


All crime categories were consolidated properly
Now we will review the Part 2 crimes, to see if any crimes were misclassified


In [20]:
# review part 2 crimes against word list for part 1 crimes

# copy the words that match into new crime category
data2['crime_category'] = data2['CrmCdDesc'].apply(lambda x: ' '.join([word for word in part1_crimes if word in x]))

# review the list of new crime categories

display(data2.crime_category.value_counts().sort_values(ascending=False).to_frame().reset_index())
print('We will explore the blank values, where no words matched and consolidate any other redundant categories')
print('Then we will consolidate other crimes where theres redundancy')


,crime_category,count
0,,305032
1,THEFT,69920
2,SEX,6091
3,VEHICLE,782
4,HUMAN TRAFFICKING SEX,446
5,HUMAN TRAFFICKING,112


We will explore the blank values, where no words matched and consolidate any other redundant categories
Then we will consolidate other crimes where theres redundancy


In [21]:
# explore the blank values
print('There are {} crime categories that are not showing up in the word list for part 1 crimes '.format(data2.loc[data2['crime_category']=='']['CrmCdDesc'].value_counts().sort_values(ascending=False).to_frame().reset_index().shape[0]))
display(data2.loc[data2['crime_category']=='']['CrmCdDesc'].value_counts()[:20].sort_values(ascending=False).to_frame().reset_index())
print('We checked all the categories and they appear to be classified properly')

There are 68 crime categories that are not showing up in the word list for part 1 crimes 


,CrmCdDesc,count
0,BATTERY - SIMPLE ASSAULT,73759
1,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",56525
2,INTIMATE PARTNER - SIMPLE ASSAULT,46165
3,VANDALISM - MISDEAMEANOR ($399 OR UNDER),24064
4,CRIMINAL THREATS - NO WEAPON DISPLAYED,18975
5,TRESPASSING,14495
6,VIOLATION OF RESTRAINING ORDER,11626
7,"LETTERS, LEWD - TELEPHONE CALLS, LEWD",7813
8,OTHER MISCELLANEOUS CRIME,6689
9,VIOLATION OF COURT ORDER,6294


We checked all the categories and they appear to be classified properly


In [22]:
# consolidate HUMAN TRAFFICKING crimes
for i in ['HUMAN TRAFFICKING SEX','HUMAN TRAFFICKING']:
    data2.loc[data2['CrmCdDesc']==i, 'crime_category'] = 'HUMAN TRAFFICKING'

# create df for human trafficking crimes
data2_ht = data2[data2['crime_category']=='HUMAN TRAFFICKING']

# merge df with data1
data1 = pd.concat([data1, data2_ht], axis=0)

# review the remaining crimes
display(data1.crime_category.value_counts().sort_values(ascending=False).to_frame().reset_index())

,crime_category,count
0,MOTOR VEHICLE THEFT,177078
1,THEFT,123681
2,BURGLARY,119115
3,AGGRAVATED ASSAULT,82869
4,ROBBERY,36453
5,RAPE,6496
6,ARSON,2515
7,HOMICIDE,1542
8,HUMAN TRAFFICKING,112


In [23]:
# save to CSV 
data1.to_csv('../data/crime_data_cleaned.csv', index=False)
print('Data has been cleaned and saved to crime_data_cleaned.csv')

Data has been cleaned and saved to crime_data_cleaned.csv
